In [ ]:
# 🛠 Step 1: Upload your CSV file
from google.colab import files
uploaded = files.upload()
# 🛠 Step 2: Install required library
!pip install plotly
# 🛠 Step 3: Import libraries
import pandas as pd
from datetime import datetime
import plotly.express as px
# 🛠 Step 4: Load and process data
# Make sure the uploaded file name matches exactly
df = pd.read_csv('healthcare_dataset.csv')

# Convert 'Date of Admission' and 'Discharge Date' to datetime
df["Date of Admission"] = pd.to_datetime(df["Date of Admission"])
df["Discharge Date"] = pd.to_datetime(df["Discharge Date"])

# Get today's date
today = pd.to_datetime(datetime.today().date())

# Determine whether the bed is occupied
df["Occupied"] = df.apply(lambda x: x["Date of Admission"] <= today <= x["Discharge Date"], axis=1)
# 🛠 Step 5: Calculate Occupancy
rooms = df["Room Number"].unique()
occupied_rooms = df[df["Occupied"]]["Room Number"].unique()
available_rooms = [room for room in rooms if room not in occupied_rooms]

print(f"🏥 Total Beds: {len(rooms)}")
print(f"🔴 Occupied Beds: {len(occupied_rooms)}")
print(f"🟢 Available Beds: {len(available_rooms)}")
# 🛠 Step 6: Visualize Bed Occupancy
status_df = pd.DataFrame({
    "Status": ["Occupied", "Available"],
    "Count": [len(occupied_rooms), len(available_rooms)]
})

fig = px.pie(status_df, names="Status", values="Count", title="Hospital Bed Occupancy Status")
fig.show()


Saving healthcare_dataset.csv to healthcare_dataset.csv
🏥 Total Beds: 400
🔴 Occupied Beds: 0
🟢 Available Beds: 400


In [ ]:
! pip install torch torchvision torchaudio
!pip install opencv-python
!pip install streamlit
!pip install pandas
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
# hospital_bed_monitoring_system.py

import cv2
import torch
import time
import streamlit as st
import threading
import pandas as pd

# -------------------------------
# Bed Detector Class
# -------------------------------

class BedDetector:
    def __init__(self, model_path):
        print("[INFO] Loading YOLO model...")
        self.model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path, force_reload=True)
        print("[INFO] Model loaded successfully.")

    def detect_beds(self, frame):
        """
        Detect beds in the given frame using YOLO model
        Returns DataFrame of detected beds
        """
        results = self.model(frame)
        detections = results.pandas().xyxy[0]
        beds = detections[detections['name'] == 'bed']  # Only select 'bed' detections
        return beds

# -------------------------------
# Dashboard Class (Streamlit)
# -------------------------------

class BedDashboard:
    def __init__(self):
        st.set_page_config(page_title="Hospital Bed Monitoring", layout="wide")
        self.placeholder = st.empty()

    def update(self, total_beds, occupied_beds, frame=None):
        empty_beds = total_beds - occupied_beds

        with self.placeholder.container():
            st.title("🏥 Real-Time Hospital Bed Monitoring")
            st.markdown("---")
            st.metric(label="Total Beds", value=total_beds)
            st.metric(label="Occupied Beds", value=occupied_beds)
            st.metric(label="Empty Beds", value=empty_beds)
            st.markdown("---")

            if frame is not None:
                st.image(frame, channels="BGR", caption="Live Hospital Feed")

# -------------------------------
# Logger Class
# -------------------------------

class BedLogger:
    def __init__(self):
        self.logs = []

    def add_log(self, timestamp, total_beds, occupied_beds, empty_beds):
        self.logs.append({
            "Time": timestamp,
            "Total Beds": total_beds,
            "Occupied Beds": occupied_beds,
            "Empty Beds": empty_beds
        })

    def save_logs(self, filename="bed_logs.csv"):
        df = pd.DataFrame(self.logs)
        df.to_csv(filename, index=False)
        print(f"[INFO] Logs saved to {filename}")

# -------------------------------
# Main Monitoring System Class
# -------------------------------

class HospitalBedMonitoringSystem:
    def __init__(self, model_path, video_source=0, total_beds=10):
        self.detector = BedDetector(model_path)
        self.dashboard = BedDashboard()
        self.logger = BedLogger()
        self.video_source = video_source
        self.total_beds = total_beds
        self.running = True

    def start_monitoring(self):
        cap = cv2.VideoCapture(self.video_source)

        if not cap.isOpened():
            print("[ERROR] Cannot open video source.")
            return

        print("[INFO] Starting Monitoring System...")

        while self.running:
            ret, frame = cap.read()
            if not ret:
                print("[WARNING] Frame not received. Exiting...")
                break

            # Detect beds
            beds_detected = self.detector.detect_beds(frame)
            occupied_beds = len(beds_detected)

            # Draw boxes on detected beds
            for idx, row in beds_detected.iterrows():
                x1, y1, x2, y2 = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, "Occupied", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

            # Update Dashboard
            self.dashboard.update(self.total_beds, occupied_beds, frame)

            # Log information
            timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
            self.logger.add_log(timestamp, self.total_beds, occupied_beds, self.total_beds - occupied_beds)

            # Sleep a little to save CPU
            time.sleep(0.5)

        cap.release()
        cv2.destroyAllWindows()

    def stop_monitoring(self):
        self.running = False
        self.logger.save_logs()

# -------------------------------
# Start everything (Threading for Streamlit)
# -------------------------------

def run_system():
    model_path = 'models/yolo_weights/best.pt'  # Change this to your model path
    total_beds_in_hospital = 10  # Change this according to your hospital size
    video_source = 0  # Camera index or video file path

    system = HospitalBedMonitoringSystem(model_path, video_source, total_beds_in_hospital)

    # Use threading so Streamlit and OpenCV don't block each other
    monitor_thread = threading.Thread(target=system.start_monitoring)
    monitor_thread.start()

    # If you want to stop manually (button)
    stop_button = st.button("Stop Monitoring")

    if stop_button:
        system.stop_monitoring()

if __name__ == "__main__":
    run_system()


[INFO] Loading YOLO model...


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2025-4-28 Python-3.11.12 torch-2.6.0+cu124 CPU



Exception: [Errno 2] No such file or directory: 'models/yolo_weights/best.pt'. Cache may be out of date, try `force_reload=True` or see https://docs.ultralytics.com/yolov5/tutorials/pytorch_hub_model_loading for help.